In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

In [2]:
Japan = pd.read_csv('JPN.csv')
print(Japan.info())     # Overview of column types
print(Japan.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          40000 non-null  object
 1   CURR_AGE    40000 non-null  int64 
 2   GENDER      40000 non-null  object
 3   ANN_INCOME  40000 non-null  object
 4   AGE_CAR     40000 non-null  int64 
 5   PURCHASE    40000 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.8+ MB
None
ID            0
CURR_AGE      0
GENDER        0
ANN_INCOME    0
AGE_CAR       0
PURCHASE      0
dtype: int64


In [3]:
Japan['ANN_INCOME'] = Japan['ANN_INCOME'].str.replace(',', '').astype(float)

In [4]:
Japan['GENDER'] = Japan['GENDER'].map({'M': 0, 'F': 1})

In [5]:
print(Japan.head())     # See first few rows
print(Japan.describe()) # For numerical stats

           ID  CURR_AGE  GENDER  ANN_INCOME  AGE_CAR  PURCHASE
0  00001Q15YJ        50       0    445344.0      439         0
1  00003I71CQ        35       0    107634.0      283         0
2  00003N47FS        59       1    502787.0      390         1
3  00005H41DE        43       0    585664.0      475         0
4  00007E17UM        39       1    705723.0      497         1
          CURR_AGE        GENDER     ANN_INCOME       AGE_CAR      PURCHASE
count  40000.00000  40000.000000   40000.000000  40000.000000  40000.000000
mean      44.99745      0.442875  359398.878225    359.080250      0.575775
std       11.82008      0.496732  175109.260472    203.063724      0.494231
min       25.00000      0.000000   70089.000000      1.000000      0.000000
25%       35.00000      0.000000  219766.000000    235.000000      0.000000
50%       45.00000      0.000000  337657.000000    331.000000      1.000000
75%       55.00000      1.000000  464260.750000    444.000000      1.000000
max       65.0

In [6]:
India = pd.read_csv("IN.csv")
print(India.info())     # Overview of column types
print(India.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          70000 non-null  object
 1   CURR_AGE    70000 non-null  int64 
 2   GENDER      70000 non-null  object
 3   ANN_INCOME  70000 non-null  object
 4   DT_MAINT    70000 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.7+ MB
None
ID            0
CURR_AGE      0
GENDER        0
ANN_INCOME    0
DT_MAINT      0
dtype: int64


In [7]:
India['ANN_INCOME'] = India['ANN_INCOME'].str.replace(',', '').astype(float)
India['GENDER'] = India['GENDER'].map({'M': 0, 'F': 1})

In [8]:
#As age car is not mentioned
# Step 1: Convert DT_MAINT to datetime
India['DT_MAINT'] = pd.to_datetime(India['DT_MAINT'], errors='coerce')

# Step 2: Define reference date
ref_date = pd.to_datetime('2019-07-01')

# Step 3: Calculate difference in days
India['AGE_CAR'] = (ref_date - India['DT_MAINT']).dt.days.round(2)

In [9]:
India.drop('DT_MAINT', axis=1, inplace=True)

In [10]:
print(India.head())     # See first few rows
print(India.describe()) # For numerical stats

           ID  CURR_AGE  GENDER  ANN_INCOME  AGE_CAR
0  20710B05XL        54       0   1425390.0      437
1  89602T51HX        47       0   1678954.0      388
2  70190Z52IP        60       0    931624.0      700
3  25623V15MU        55       1   1106320.0      700
4  36230I68CE        32       1    748465.0      155
           CURR_AGE        GENDER    ANN_INCOME       AGE_CAR
count  70000.000000  70000.000000  7.000000e+04  70000.000000
mean      44.995314      0.499586  1.148679e+06    367.326057
std       11.822122      0.500003  3.994505e+05    241.999792
min       25.000000      0.000000  3.000330e+05      1.000000
25%       35.000000      0.000000  8.568238e+05    189.000000
50%       45.000000      0.000000  1.125152e+06    340.000000
75%       55.000000      1.000000  1.438676e+06    473.000000
max       65.000000      1.000000  1.999989e+06   1020.000000


In [11]:
def assign_segment(days):
    if pd.isnull(days):
        return None
    elif days < 200:
        return 1
    elif 200 <= days <= 360:
        return 2
    elif 360 < days <= 500:
        return 3
    else:
        return 4

Japan['AGE_CAR_SEGMENT'] = Japan['AGE_CAR'].apply(assign_segment)
India['AGE_CAR_SEGMENT'] = India['AGE_CAR'].apply(assign_segment)

In [12]:
print(Japan['AGE_CAR_SEGMENT'].value_counts(dropna=False).sort_index())
print(India['AGE_CAR_SEGMENT'].value_counts(dropna=False).sort_index())

AGE_CAR_SEGMENT
1     6459
2    16545
3    11697
4     5299
Name: count, dtype: int64
AGE_CAR_SEGMENT
1    18496
2    18994
3    18691
4    13819
Name: count, dtype: int64


In [13]:
#Model :
#1.Logistic Regression was selected as a baseline model due to its interpretability and suitability for binary classification.
#2.AGE_CAR was bucketed into categorical segments to reflect its non-linear impact on purchase behavior.
#3.One-Hot Encoding was used to convert categorical data for modeling.

# Select features
features = ['CURR_AGE', 'GENDER', 'ANN_INCOME', 'AGE_CAR_SEGMENT']
target = 'PURCHASE'

# One-Hot Encode 'GENDER' and 'AGE_CAR_SEGMENT'
encoder = OneHotEncoder(drop='first', sparse=False)
Japan_encoded = pd.DataFrame(encoder.fit_transform(Japan[['GENDER', 'AGE_CAR_SEGMENT']]))
Japan_encoded.columns = encoder.get_feature_names_out(['GENDER', 'AGE_CAR_SEGMENT'])
Japan_final = pd.concat([Japan[features], Japan_encoded], axis=1).drop(['GENDER', 'AGE_CAR_SEGMENT'], axis=1)
Japan_final[target] = Japan[target]

/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
X_Japan = Japan_final.drop('PURCHASE', axis=1)
Y_Japan = Japan_final['PURCHASE'].round(1)

X_Japan_train, X_Japan_test, Y_Japan_train, Y_Japan_test = train_test_split(X_Japan, Y_Japan, test_size=0.3, random_state=42)

In [15]:
model_japan = LogisticRegression()
model_japan.fit(X_Japan_train, Y_Japan_train)

LogisticRegression()

In [16]:
Y_Japan_pred = model_japan.predict(X_Japan_test)

In [17]:
coefficients = pd.DataFrame({
    'Feature': X_Japan.columns,
    'Coefficient': model_japan.coef_[0]
})
print(coefficients.sort_values(by='Coefficient', ascending=False))

             Feature   Coefficient
1         ANN_INCOME  1.038413e-06
4  AGE_CAR_SEGMENT_3  6.725100e-12
0           CURR_AGE  4.743245e-12
5  AGE_CAR_SEGMENT_4  3.699234e-12
2           GENDER_1 -8.993006e-13
3  AGE_CAR_SEGMENT_2 -6.567254e-12


In [18]:
# Add interpretation
# A positive coefficient for "ANN_INCOME" suggests higher-income individuals are more likely to purchase.
# A negative coefficient for "AGE_CAR_SEGMENT_4" implies that very old cars are associated with lower purchase probability.

for _, row in coefficients.iterrows():
    direction = "positive" if row['Coefficient'] > 0 else "negative"
    print(f"A 1-unit increase in {row['Feature']} has a {direction} effect on the probability of purchasing.")

A 1-unit increase in CURR_AGE has a positive effect on the probability of purchasing.
A 1-unit increase in ANN_INCOME has a positive effect on the probability of purchasing.
A 1-unit increase in GENDER_1 has a negative effect on the probability of purchasing.
A 1-unit increase in AGE_CAR_SEGMENT_2 has a negative effect on the probability of purchasing.
A 1-unit increase in AGE_CAR_SEGMENT_3 has a positive effect on the probability of purchasing.
A 1-unit increase in AGE_CAR_SEGMENT_4 has a positive effect on the probability of purchasing.


In [19]:
Y_Japan_pred = model_japan.predict(X_Japan_test)
Y_Japan_prob = model_japan.predict_proba(X_Japan_test)[:, 1]
print("Accuracy:", accuracy_score(Y_Japan_test, Y_Japan_pred))
print("ROC-AUC Score:", roc_auc_score(Y_Japan_test, Y_Japan_prob))
print(confusion_matrix(Y_Japan_test, Y_Japan_pred))
print(classification_report(Y_Japan_test, Y_Japan_pred))

Accuracy: 0.58225
ROC-AUC Score: 0.6125372300231792
[[   0 5013]
 [   0 6987]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5013
           1       0.58      1.00      0.74      6987

    accuracy                           0.58     12000
   macro avg       0.29      0.50      0.37     12000
weighted avg       0.34      0.58      0.43     12000



/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [20]:
# Encoding India dataset similarly
India_encoded = pd.DataFrame(encoder.transform(India[['GENDER', 'AGE_CAR_SEGMENT']]))
India_encoded.columns = encoder.get_feature_names_out(['GENDER', 'AGE_CAR_SEGMENT'])

India_final = pd.concat([India[features], India_encoded], axis=1).drop(['GENDER', 'AGE_CAR_SEGMENT'], axis=1)

# Predicting probabilities
India['PURCHASE_PROB'] = model_japan.predict_proba(India_final).round(2)[:, 1]
India['PREDICTED_PURCHASE'] = (India['PURCHASE_PROB'] >= 0.5).astype(int)

# Count of potential customers
potential_buyers = India['PREDICTED_PURCHASE'].sum()
print("Potential customers in India:", potential_buyers)


Potential customers in India: 70000


In [21]:
# Evaluating the Model
print("Accuracy:", accuracy_score(Y_Japan_test, Y_Japan_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_Japan_test, Y_Japan_pred))
print("Classification Report:\n", classification_report(Y_Japan_test, Y_Japan_pred))
print("ROC AUC Score:", roc_auc_score(Y_Japan_test, model_japan.predict_proba(X_Japan_test)[:, 1]))

Accuracy: 0.58225
Confusion Matrix:
 [[   0 5013]
 [   0 6987]]
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      5013
           1       0.58      1.00      0.74      6987

    accuracy                           0.58     12000
   macro avg       0.29      0.50      0.37     12000
weighted avg       0.34      0.58      0.43     12000

ROC AUC Score: 0.6125372300231792


/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [22]:
# Interpreting coefficients
for _, row in coefficients.iterrows():
    direction = "positive" if row['Coefficient'] > 0 else "negative"
    print(f"A 1-unit increase in {row['Feature']} has a {direction} effect on the probability of purchasing.")

A 1-unit increase in CURR_AGE has a positive effect on the probability of purchasing.
A 1-unit increase in ANN_INCOME has a positive effect on the probability of purchasing.
A 1-unit increase in GENDER_1 has a negative effect on the probability of purchasing.
A 1-unit increase in AGE_CAR_SEGMENT_2 has a negative effect on the probability of purchasing.
A 1-unit increase in AGE_CAR_SEGMENT_3 has a positive effect on the probability of purchasing.
A 1-unit increase in AGE_CAR_SEGMENT_4 has a positive effect on the probability of purchasing.


In [23]:
Japan.to_csv("cleaned_japan.csv", index=False)
India.to_csv("cleaned_india_with_preds.csv", index=False)

In [24]:
# Combining for Tableau
Japan['COUNTRY'] = 'JAPAN'
India['COUNTRY'] = 'INDIA'

# Combining datasets
Combined = pd.concat([Japan, India], ignore_index=True)

# Converting PURCHASE to integer
Combined['PURCHASE'] = Combined['PURCHASE'].fillna(0).astype(int)

# Converting PREDICTED_PURCHASE to integer 
Combined['PREDICTED_PURCHASE'] = Combined['PREDICTED_PURCHASE'].fillna(0).astype(int)

#Converting Gender into text form
# For combined dataset
Combined['GENDER'] = Combined['GENDER'].replace({0: 'M', 1: 'F'})

# Saving combined dataset
Combined.to_csv("Combined.csv", index=False)